In [7]:
using DrWatson
@quickactivate "MEngProject"
using MEngProject

In [8]:
using CUDA, PyPlot, NNlib,  ImageFiltering, Images, MEngProject, MEngProject.LamKernels, MEngProject.Laminart, MEngProject.Utils, BenchmarkTools, Test

In [9]:
function reshape2d_4d(img::AbstractArray)
    out = reshape(img, size(img)[1], size(img)[2], 1, 1)
end

reshape2d_4d (generic function with 1 method)

In [10]:
img = convert(Array{Float32,2}, load(datadir("Iine_100_100_gs.png")));
p = Laminart.kernels(img, Parameters.parameters);
p = Laminart.add_I_u_p(img, p);

In [25]:
img_ = CuArray(img)
img_ = reshape2d_4d(img_)
p_ = LaminartGPU.kernels(img_, Parameters.parameters);
# p_ = LaminartGPU.add_I_u_p(img_, p_);

UndefVarError: UndefVarError: W_temp not defined

In [11]:
img_ = CuArray(img)

100×100 CuArray{Float32,2,Nothing}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.

In [24]:
img_[:,:,1,1]

100×100 CuArray{Float32,2,Nothing}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.

In [12]:
    C_A_temp = reshape(
        CuArray{eltype(img)}(undef, p.C_AB_l, p.C_AB_l * p.K),
        p.C_AB_l,
        p.C_AB_l,
        p.K, 1,1
    )
    C_B_temp = copy(C_A_temp)
    H_temp = reshape(
        CuArray{eltype(img)}(undef, p.H_l, p.H_l * p.K),
        p.H_l,
        p.H_l,
        p.K,1,1
    )
    T_temp = reshape(CuArray{eltype(img)}(undef, 1, 1 * p.K), 1, 1, p.K,1,1)     #ijk,  1x1xk,   ijk
    W_temp =
        reshape(CuArray{eltype(img)}(undef, p.W_l, p.W_l * p.K * p.K), p.W_l, p.W_l, p.K, p.K,1,1)     #ijk,  1x1xk,   ijk
    for k ∈ 1:p.K
        θ = π * (k - 1) / p.K
        C_A_temp[:, :, k,1,1] = CuArray(reflect(centered(LamKernels.kern_A(p.σ_2, θ))))           #ij ijk ijk
        C_B_temp[:, :, k,1,1] = Creflect(centered(LamKernels.kern_B(p.σ_2, θ)))               #ij ijk ijk
        H_temp[:, :, k,1,1] = reflect(
            p.H_fact .* LamKernels.gaussian_rot(p.H_σ_x, p.H_σ_y, θ, p.H_l),
        )  #ijk, ij for each k; ijk
        T_temp[:, :, k] = reshape([p.T_fact[k]], 1, 1)
        #todo: generalise T and W for higher K
        #         T_temp[:,:,k] = KernelFactors.gaussian(p.T_σ, p.K)
        #         for l ∈ 1:p.K
        #             W_temp[:,:,l,k] =
        #         end
    end
    W_temp[:, :, 1, 1] = reflect(
        5 .* LamKernels.gaussian_rot(3, 0.8, 0, p.W_l) .+
        LamKernels.gaussian_rot(0.4, 1, 0, p.W_l),
    )
    W_temp[:, :, 2, 2] = reflect(
        5 .* LamKernels.gaussian_rot(3, 0.8, 0, p.W_l) .+
        LamKernels.gaussian_rot(0.4, 1, π / 2, p.W_l),
    )
    W_temp[:, :, 1, 2] = reflect(relu.(
        0.2 .- LamKernels.gaussian_rot(2, 0.6, 0, p.W_l) .-
        LamKernels.gaussian_rot(0.3, 1.2, 0, p.W_l),
    ))
    W_temp[:, :, 2, 1] = reflect(relu.(
        0.2 .- LamKernels.gaussian_rot(2, 0.6, 0, p.W_l) .-
        LamKernels.gaussian_rot(0.3, 1.2, π / 2, p.W_l),
    ))

    # todo fix W kernel
    #  W_temp[:,:,1,1] = reflect(LamKernels.gaussian_rot(3,0.8,0,19))
    #     W_temp[:,:,2,2] = reflect(LamKernels.gaussian_rot(3,0.8,0,19))
    #     W_temp[:,:,1,2] = reflect(LamKernels.gaussian_rot(3,0.8,0,19))
    #     W_temp[:,:,2,1] = reflect(LamKernels.gaussian_rot(3,0.8,0,19))

    # todo: fix range of W H
    #     W_range = -(p.W_size-1)/2:(p.W_size-1)/2
    #     H_range = -(p.H_size-1)/2:(p.H_size-1)/2
    W_range = -9:9
    H_range = -9:9

    temp_out = (
        k_gauss_1 = reflect(Kernel.gaussian(p.σ_1)),
        k_gauss_2 = reflect(Kernel.gaussian(p.σ_2)),
        k_C_A = C_A_temp,
        k_C_B = C_B_temp,
        k_W_p = W_temp,
        k_W_m = W_temp,
        # k_W_m = OffsetArray(W_temp, W_range, W_range, 1:p.K, 1:p.K),
        k_H = H_temp,
        # k_H = OffsetArray(H_temp, H_range, H_range, 1:p.K),
        k_T_p = T_temp,
        k_T_m = (p.T_p_m .* T_temp),
        k_T_p_v2 = (p.T_v2_fact .* T_temp),
        k_T_m_v2 = (p.T_v2_fact .* p.T_p_m .* T_temp),
        dim_i = size(img)[1],
        dim_j = size(img)[2],
        x_V2 = reshape(
            zeros(typeof(img[1, 1]), size(img)[1], size(img)[2] * p.K),
            size(img)[1],
            size(img)[2],
            p.K,
        ),
    )
    return merge(p, temp_out)
end

BoundsError: BoundsError: attempt to access 5×5 Array{Float64,2} at index [-2:2, -2:2]

In [ ]:
   C_A_temp = reshape(
        Array{eltype(img)}(undef, p.C_AB_l, p.C_AB_l * p.K),
        p.C_AB_l,
        p.C_AB_l,
        p.K, 1,1
    )
 C_B_temp = copy(C_A_temp)
 H_temp = reshape(
        Array{eltype(img)}(undef, p.H_l, p.H_l * p.K),
        p.H_l,
        p.H_l,
        p.K,1,1
    )
    W_temp =
        reshape(Array{eltype(img)}(undef, p.W_l, p.W_l * p.K * p.K), p.W_l, p.W_l, p.K, p.K,1,1) 

In [13]:
   C_A_temp = reshape(
        Array{eltype(img)}(undef, p.C_AB_l, p.C_AB_l * p.K),
        p.C_AB_l,
        p.C_AB_l,
        p.K, 1,1
    )

 T_temp = reshape(Array{eltype(img)}(undef, 1, 1 * p.K), 1, 1, p.K,1,1)     #ijk,  1x1xk,   ijk

    for k ∈ 1:p.K
        θ = π * (k - 1.0f0) / p.K
        C_A_temp[:, :, k,1,1] = LamKernels.kern_A(p.σ_2, θ)           #ij ijk ijk
        C_B_temp[:, :, k,1,1] = LamKernels.kern_B(p.σ_2, θ)               #ij ijk ijk
        H_temp[:, :, k,1,1] = p.H_fact .* LamKernels.gaussian_rot(p.H_σ_x, p.H_σ_y, θ, p.H_l)  #ijk, ij for each k; ijk
        T_temp[1, 1, k,1,1] = p.T_fact[k]
        #todo: generalise T and W for higher K
        #         T_temp[:,:,k] = KernelFactors.gaussian(p.T_σ, p.K)
        #         for l ∈ 1:p.K
        #             W_temp[:,:,l,k] =
        #         end
    end

    W_temp[:, :, 1, 1, 1, 1] = 
        5f0 .* LamKernels.gaussian_rot(3f0, 0.8f0, 0f0, p.W_l) .+
        LamKernels.gaussian_rot(0.4f0, 1f0, 0f0, p.W_l)
    W_temp[:, :, 2, 2, 1, 1] = 
        5f0 .* LamKernels.gaussian_rot(3f0, 0.8f0, 0f0, p.W_l) .+
        LamKernels.gaussian_rot(0.4f0, 1f0, π / 2f0, p.W_l)
    W_temp[:, :, 1, 2, 1, 1] = relu.(
        0.2f0 .- LamKernels.gaussian_rot(2f0, 0.6f0, 0f0, p.W_l) .-
        LamKernels.gaussian_rot(0.3f0, 1.2f0, 0f0, p.W_l))
    W_temp[:, :, 2, 1, 1, 1] = relu.(
        0.2f0 .- LamKernels.gaussian_rot(2f0, 0.6f0, 0f0, p.W_l) .-
        LamKernels.gaussian_rot(0.3f0, 1.2f0, π / 2f0, p.W_l))

temp_out = (
        k_gauss_1 = CuArray(reshape2d_4d(Kernel.gaussian(p.σ_1))),
        k_gauss_2 = CuArray(reshape2d_4d(Kernel.gaussian(p.σ_2))),
        k_C_A = CuArray(C_A_temp),
        k_C_B = CuArray(C_B_temp),
        k_W_p = CuArray(W_temp),
        k_W_m = CuArray(W_temp),
        k_H = CuArray(H_temp),
        k_T_p = CuArray(T_temp),
        k_T_m = CuArray((p.T_p_m .* T_temp)),
        k_T_p_v2 = CuArray((p.T_v2_fact .* T_temp)),
        k_T_m_v2 = CuArray((p.T_v2_fact .* p.T_p_m .* T_temp)),
        dim_i = size(img)[1],
        dim_j = size(img)[2],
        x_V2 = reshape(CUDA.zeros(typeof(img[1, 1]), size(img)[1], size(img)[2] * p.K), size(img)[1], size(img)[2],p.K,1,1),)
merge(p, temp_out)
end

5×5×2×1×1 Array{Float32,5}:
[:, :, 1, 1, 1] =
 -2.48973f-23   4.5687f-41   -2.48976f-23   4.5687f-41   -2.48983f-23
  4.5687f-41   -2.48975f-23   4.5687f-41   -2.4898f-23    4.5687f-41
 -2.48974f-23   4.5687f-41   -2.48977f-23   4.5687f-41   -2.48984f-23
  4.5687f-41   -2.48975f-23   4.5687f-41   -2.48982f-23   4.5687f-41
 -2.48974f-23   4.5687f-41   -2.48978f-23   4.5687f-41   -2.48984f-23

[:, :, 2, 1, 1] =
  4.5687f-41   -2.48986f-23   4.5687f-41   -2.48989f-23   4.5687f-41
 -2.48985f-23   4.5687f-41   -2.48988f-23   4.5687f-41   -3.2531f-23
  4.5687f-41   -2.48987f-23   4.5687f-41   -3.25309f-23   4.5687f-41
 -2.48986f-23   4.5687f-41   -2.48988f-23   4.5687f-41   -2.4899f-23
  4.5687f-41   -2.48987f-23   4.5687f-41   -2.48989f-23   4.5687f-41

In [14]:
 C_B_temp = copy(C_A_temp)
   

5×5×2×1×1 Array{Float32,5}:
[:, :, 1, 1, 1] =
 -2.48973f-23   4.5687f-41   -2.48976f-23   4.5687f-41   -2.48983f-23
  4.5687f-41   -2.48975f-23   4.5687f-41   -2.4898f-23    4.5687f-41
 -2.48974f-23   4.5687f-41   -2.48977f-23   4.5687f-41   -2.48984f-23
  4.5687f-41   -2.48975f-23   4.5687f-41   -2.48982f-23   4.5687f-41
 -2.48974f-23   4.5687f-41   -2.48978f-23   4.5687f-41   -2.48984f-23

[:, :, 2, 1, 1] =
  4.5687f-41   -2.48986f-23   4.5687f-41   -2.48989f-23   4.5687f-41
 -2.48985f-23   4.5687f-41   -2.48988f-23   4.5687f-41   -3.2531f-23
  4.5687f-41   -2.48987f-23   4.5687f-41   -3.25309f-23   4.5687f-41
 -2.48986f-23   4.5687f-41   -2.48988f-23   4.5687f-41   -2.4899f-23
  4.5687f-41   -2.48987f-23   4.5687f-41   -2.48989f-23   4.5687f-41

In [15]:
 H_temp = reshape(
        Array{eltype(img)}(undef, p.H_l, p.H_l * p.K),
        p.H_l,
        p.H_l,
        p.K,1,1
    )

19×19×2×1×1 Array{Float32,5}:
[:, :, 1, 1, 1] =
 -8.86134f-19   4.5687f-41   -9.15323f-19  …   4.5687f-41   -2.30124f-16
  4.5687f-41   -8.73127f-19   4.5687f-41      -8.89105f-19   4.5687f-41
 -8.73087f-19   4.5687f-41   -9.15337f-19      4.5687f-41   -8.89131f-19
  4.5687f-41   -9.81183f-19   4.5687f-41      -8.89108f-19   4.5687f-41
 -8.73091f-19   4.5687f-41   -9.15346f-19      4.5687f-41   -9.8476f-19
  4.5687f-41   -9.90716f-19   4.5687f-41   …  -8.89112f-19   4.5687f-41
 -8.73094f-19   4.5687f-41   -9.81418f-19      4.5687f-41   -8.89138f-19
  4.5687f-41   -9.90719f-19   4.5687f-41      -8.89115f-19   4.5687f-41
 -9.80988f-19   4.5687f-41   -9.81435f-19      4.5687f-41   -8.89141f-19
  4.5687f-41   -9.90722f-19   4.5687f-41      -9.84615f-19   4.5687f-41
 -9.90713f-19   4.5687f-41   -9.81451f-19  …   4.5687f-41   -8.89145f-19
  4.5687f-41   -8.73137f-19   4.5687f-41      -2.30109f-16   4.5687f-41
 -8.73104f-19   4.5687f-41   -1.18508f25       4.5687f-41   -9.90726f-19
  4.5687f-

In [16]:
 T_temp = reshape(Array{eltype(img)}(undef, 1, 1 * p.K), 1, 1, p.K,1,1)     #ijk,  1x1xk,   ijk

1×1×2×1×1 Array{Float32,5}:
[:, :, 1, 1, 1] =
 -2.447092f-23

[:, :, 2, 1, 1] =
 4.5687f-41

In [17]:
    W_temp =
        reshape(Array{eltype(img)}(undef, p.W_l, p.W_l * p.K * p.K), p.W_l, p.W_l, p.K, p.K,1,1) 

19×19×2×2×1×1 Array{Float32,6}:
[:, :, 1, 1, 1, 1] =
   0.0            0.0            2.98998f-33  …  NaN              3.0f-45
   0.0            0.0            0.0               0.0            0.0
   2.98979f-33    0.0           -4.55969f32        0.0            0.0
   0.0          NaN              4.5687f-41        1.05101f-33    0.0
  -4.55969f32   NaN              3.0f-45           0.0          NaN
   4.5687f-41     0.0            0.0          …   -4.55969f32   NaN
   3.0f-45        0.0            0.0               4.5687f-41     0.0
   0.0            2.98992f-33    0.0               3.0f-45        0.0
   0.0            0.0          NaN                 0.0            4.2117f-32
   0.0           -4.55969f32   NaN                 0.0            0.0
 NaN              4.5687f-41     0.0          …    0.0           -4.55969f32
 NaN              3.0f-45        0.0             NaN              4.5687f-41
   0.0            0.0            2.9901f-33      NaN              3.0f-45
   0.0      

In [18]:
    #ijk,  1x1xk,   ijk
    for k ∈ 1:p.K
        θ = π * (k - 1.0f0) / p.K
        C_A_temp[:, :, k,1,1] = LamKernels.kern_A(p.σ_2, θ)           #ij ijk ijk
        C_B_temp[:, :, k,1,1] = LamKernels.kern_B(p.σ_2, θ)               #ij ijk ijk
        H_temp[:, :, k,1,1] = p.H_fact .* LamKernels.gaussian_rot(p.H_σ_x, p.H_σ_y, θ, p.H_l)  #ijk, ij for each k; ijk
        T_temp[1, 1, k,1,1] = p.T_fact[k]
        #todo: generalise T and W for higher K
        #         T_temp[:,:,k] = KernelFactors.gaussian(p.T_σ, p.K)
        #         for l ∈ 1:p.K
        #             W_temp[:,:,l,k] =
        #         end
    end

In [19]:
    W_temp[:, :, 1, 1, 1, 1] = 
        5f0 .* LamKernels.gaussian_rot(3f0, 0.8f0, 0f0, p.W_l) .+
        LamKernels.gaussian_rot(0.4f0, 1f0, 0f0, p.W_l)
    W_temp[:, :, 2, 2, 1, 1] = 
        5f0 .* LamKernels.gaussian_rot(3f0, 0.8f0, 0f0, p.W_l) .+
        LamKernels.gaussian_rot(0.4f0, 1f0, π / 2f0, p.W_l)
    W_temp[:, :, 1, 2, 1, 1] = relu.(
        0.2f0 .- LamKernels.gaussian_rot(2f0, 0.6f0, 0f0, p.W_l) .-
        LamKernels.gaussian_rot(0.3f0, 1.2f0, 0f0, p.W_l))
    W_temp[:, :, 2, 1, 1, 1] = relu.(
        0.2f0 .- LamKernels.gaussian_rot(2f0, 0.6f0, 0f0, p.W_l) .-
        LamKernels.gaussian_rot(0.3f0, 1.2f0, π / 2f0, p.W_l))

19×19 OffsetArray(::Array{Float64,2}, -9:9, -9:9) with eltype Float64 with indices -9:9×-9:9:
 0.2  0.2  0.2  0.2  0.2  0.2  0.2  …  0.2  0.2  0.2  0.2  0.2  0.2  0.2
 0.2  0.2  0.2  0.2  0.2  0.2  0.2     0.2  0.2  0.2  0.2  0.2  0.2  0.2
 0.2  0.2  0.2  0.2  0.2  0.2  0.2     0.2  0.2  0.2  0.2  0.2  0.2  0.2
 0.2  0.2  0.2  0.2  0.2  0.2  0.2     0.2  0.2  0.2  0.2  0.2  0.2  0.2
 0.2  0.2  0.2  0.2  0.2  0.2  0.2     0.2  0.2  0.2  0.2  0.2  0.2  0.2
 0.2  0.2  0.2  0.2  0.2  0.2  0.2  …  0.2  0.2  0.2  0.2  0.2  0.2  0.2
 0.2  0.2  0.2  0.2  0.2  0.2  0.2     0.2  0.2  0.2  0.2  0.2  0.2  0.2
 0.2  0.2  0.2  0.2  0.2  0.2  0.2     0.2  0.2  0.2  0.2  0.2  0.2  0.2
 0.2  0.2  0.2  0.2  0.2  0.2  0.2     0.2  0.2  0.2  0.2  0.2  0.2  0.2
 0.2  0.2  0.2  0.2  0.2  0.2  0.2     0.2  0.2  0.2  0.2  0.2  0.2  0.2
 0.2  0.2  0.2  0.2  0.2  0.2  0.2  …  0.2  0.2  0.2  0.2  0.2  0.2  0.2
 0.2  0.2  0.2  0.2  0.2  0.2  0.2     0.2  0.2  0.2  0.2  0.2  0.2  0.2
 0.2  0.2  0.2  0.2  0.2  0.2 

In [20]:
    # todo fix W kernel
    #  W_temp[:,:,1,1] = reflect(LamKernels.gaussian_rot(3,0.8,0,19))
    #     W_temp[:,:,2,2] = reflect(LamKernels.gaussian_rot(3,0.8,0,19))
    #     W_temp[:,:,1,2] = reflect(LamKernels.gaussian_rot(3,0.8,0,19))
    #     W_temp[:,:,2,1] = reflect(LamKernels.gaussian_rot(3,0.8,0,19))

  

    temp_out = (
        k_gauss_1 = CuArray(reshape2d_4d(Kernel.gaussian(p.σ_1))),
        k_gauss_2 = CuArray(reshape2d_4d(Kernel.gaussian(p.σ_2))),
        k_C_A = CuArray(C_A_temp),
        k_C_B = CuArray(C_B_temp),
        k_W_p = CuArray(W_temp),
        k_W_m = CuArray(W_temp),
        k_H = CuArray(H_temp),
        k_T_p = CuArray(T_temp),
        k_T_m = CuArray((p.T_p_m .* T_temp)),
        k_T_p_v2 = CuArray((p.T_v2_fact .* T_temp)),
        k_T_m_v2 = CuArray((p.T_v2_fact .* p.T_p_m .* T_temp)),
        dim_i = size(img)[1],
        dim_j = size(img)[2],
        x_V2 = reshape(CUDA.zeros(typeof(img[1, 1]), size(img)[1], size(img)[2] * p.K), size(img)[1], size(img)[2],p.K,1,1),)

(k_gauss_1 = Float32[0.0029690173 0.013306212 … 0.013306212 0.0029690173; 0.013306212 0.059634306 … 0.059634306 0.013306212; … ; 0.013306212 0.059634306 … 0.059634306 0.013306212; 0.0029690173 0.013306212 … 0.013306212 0.0029690173], k_gauss_2 = Float32[6.9624775f-8 2.808864f-5 … 2.808864f-5 6.9624775f-8; 2.808864f-5 0.011331766 … 0.011331766 2.808864f-5; … ; 2.808864f-5 0.011331766 … 0.011331766 2.808864f-5; 6.9624775f-8 2.808864f-5 … 2.808864f-5 6.9624775f-8], k_C_A = Float32[0.124950476 0.04048735 … -0.04048735 -0.124950476; 0.124950476 0.04048735 … -0.04048735 -0.124950476; … ; 0.124950476 0.04048735 … -0.04048735 -0.124950476; 0.124950476 0.04048735 … -0.04048735 -0.124950476]

Float32[0.12495046 0.12495046 … 0.124950476 0.12495049; 0.040487345 0.040487345 … 0.04048735 0.040487353; … ; -0.040487353 -0.04048735 … -0.040487345 -0.040487345; -0.12495049 -0.124950476 … -0.12495046 -0.12495046], k_C_B = Float32[0.124950476 0.04048735 … 0.04048735 0.124950476; 0.124950476 0.04048735 … 0

In [21]:
    merge(p, temp_out)

(K = 2, δ_v = 1.25f0, δ_c = 0.25f0, δ_m = 0.01875f0, δ_z = 0.125f0, δ_s = 2.5f0, σ_1 = 1.0f0, C_1 = 1.5f0, C_2 = 0.075f0, σ_2 = 0.5f0, γ = 10.0f0, α = 0.5f0, ϕ = 2.0f0, Γ = 0.2f0, v_21 = 0.0f0, μ = 2.0f0, ν = 1.1f0, n = 6.0f0, att = 0.0f0, η_p = 2.1f0, η_m = 1.5f0, λ = 1.5f0, a_23_ex = 3.0f0, a_23_in = 0.5f0, v12_6 = 1.0f0, v12_4 = 5.0f0, ψ = 0.5f0, C_AB_l = 5, H_σ_x = 3.0f0, H_σ_y = 4.0f0, H_fact = 5.0f0, H_l = 19, T_fact = Float32[0.87, 0.13], T_p_m = 0.302f0, T_v2_fact = 0.625f0, H_size = 19, W_size = 19, lgn_equ_u = 1.0f0, lgn_equ_A = 0.0f0, lgn_equ_B = 0.0f0, filling = "circular", W_l = 19, k_gauss_1 = Float32[0.0029690173 0.013306212 … 0.013306212 0.0029690173; 0.013306212 0.059634306 … 0.059634306 0.013306212; … ; 0.013306212 0.059634306 … 0.059634306 0.013306212; 0.0029690173 0.013306212 … 0.013306212 0.0029690173], k_gauss_2 = Float32[6.9624775f-8 2.808864f-5 … 2.808864f-5 6.9624775f-8; 2.808864f-5 0.011331766 … 0.011331766 2.808864f-5; … ; 2.808864f-5 0.011331766 … 0.01133176

In [22]:
gaussian_rot(2f0, 0.6f0, 0f0, p.W_l)

UndefVarError: UndefVarError: gaussian_rot not defined

In [81]:
using OffsetArrays
function gaussian_rot(σ_x::Real, σ_y::Real, θ::Real, l = 4*ceil(Int, max(σ_a,σ_b))+1)
    isodd(l) || throw(ArgumentError("length must be odd"))
    w = l>>1
    g = OffsetArray(fill(0.0f0, l, l), -w:w, -w:w)
    #todo add when σ_x or/and σ_y == 0
    for x ∈ -w:w, y ∈ -w:w
        g[x,y] = exp(-1f0/2f0*((((x*cos(θ)-y*sin(θ))/σ_x)^2f0)+(((x*sin(θ)+y*cos(θ))/σ_y)^2f0)))
    end
    centered(g/sum(g))
end

gaussian_rot (generic function with 2 methods)

In [33]:
img_[:,1,:,:]

100×1×1 CuArray{Float32,3,Nothing}:
[:, :, 1] =
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [60]:
T_temp = reshape(Array{eltype(img)}(undef, 1, 1 * p.K), 1, 1, p.K,1,1)

1×1×2×1×1 Array{Float32,5}:
[:, :, 1, 1, 1] =
 NaN

[:, :, 2, 1, 1] =
 NaN

In [61]:
T_temp[1, 1, 1,1,1] = p.T_fact[1]

0.87f0

In [62]:
T_temp[1, 1, 2,1,1] = p.T_fact[2]

0.13f0

In [63]:
T_temp[:,:,1,:,:]

1×1×1×1 Array{Float32,4}:
[:, :, 1, 1] =
 0.87

In [43]:
T_temp_ = reshape(Array{eltype(img)}(undef, 1, 1 * p.K), 1, 1, p.K)

1×1×2 Array{Float32,3}:
[:, :, 1] =
 -4.352956f13

[:, :, 2] =
 4.5884f-41

In [44]:
T_temp_[:, :, 1] = reshape([p.T_fact[1]], 1, 1)

1×1 Array{Float32,2}:
 0.87

In [45]:
T_temp_

1×1×2 Array{Float32,3}:
[:, :, 1] =
 0.87

[:, :, 2] =
 4.5884f-41

In [46]:
T_temp__ = reshape(Array{eltype(img)}(undef, 1, 1 * p.K), 1, 1, p.K)

1×1×2 Array{Float32,3}:
[:, :, 1] =
 -4.0566033f13

[:, :, 2] =
 4.5884f-41

In [47]:
T_temp__[:, :, 1] = p.T_fact[1]

ArgumentError: ArgumentError: indexed assignment with a single value to many locations is not supported; perhaps use broadcasting `.=` instead?

In [48]:
 p.T_fact

2-element Array{Float32,1}:
 0.87
 0.13

In [73]:
 π / 2f0

1.5707964f0